In [1]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup


# 네이버 영화 댓글 
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1

## 한페이지 댓글

In [4]:
# 1) requests 모듈 페이지 요청 => html 페이지를 텍스트로 저장 
url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1'
res = requests.get(url)
# 200 이면 성공 
print(res) 
# html 페이지를 텍스트로 저장 
html_str = res.text
print(type(html_str))
# print(html_str)

<Response [200]>
<class 'str'>


In [9]:
# 2) BeautifulSoup() 이용... html 페이지(str) => soup 객체
#  BeautifulSoup(html페이지소스, 파서(html.parser/lxml/xml))
soup = BeautifulSoup(html_str, 'html.parser')
print(type(soup))
# print(soup)

<class 'bs4.BeautifulSoup'>


In [11]:
# 3) 필요한 요소만 리스트로 저장 
# 필터링 요소 찾기 - 태그 위주 

# td_list = soup.find_all(class_='title')
td_list = soup.select('.title')

In [15]:
print('페이지당 댓글 갯수는?', len(td_list))
print('상위 첫번째 댓글 소스 확인')
td_list[0]

페이지당 댓글 갯수는? 10
상위 첫번째 댓글 소스 확인


<td class="title">
<a class="movie color_b" href="/movie/bi/mi/basic.naver?code=13558">키드</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
</div>
<br/>짧고 굵게 행복하고 사랑스럽다 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('jsyb****', 'fEM3fml7YAMBs3rbbpBaBwaTap0SMR6mTgwgM6fEtvk=', '짧고 굵게 행복하고 사랑스럽다', '17749551', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>

In [25]:
# 첫번째 요소에 필요한 텍스트만 추출 
# title = td_list[0].find('a').text
title = td_list[0].select_one('a').text # 영화 제목
score = td_list[0].select_one('em').text  # 평점 
# .next_sibling 요소의 다음 형제 요소 찾기 
# br 태그 요소의 다음 요소를 찾아서 공백등 제거 후 저장해줘 
review = td_list[0].select_one('br').next_sibling.strip()

base_url = 'https://movie.naver.com'
link = base_url + td_list[0].select_one('a')['href']
print(title, score, review, link)

키드 10 짧고 굵게 행복하고 사랑스럽다 https://movie.naver.com/movie/bi/mi/basic.naver?code=13558


In [38]:
# 날짜와 작성자 텍스트 추출 
a_list = soup.select('.author')
len(a_list)
print(a_list[0].text, a_list[0].next_sibling.next_sibling)
print()
print(a_list[0])
print(a_list[0].next_sibling)
print(a_list[0].next_sibling.next_sibling)

jsyb**** 21.10.20

<a class="author" href="javascript:find_list('nickname','17749551', 'after');">jsyb****</a>
<br/>
21.10.20


In [30]:
# 모든 요소를 찾아서 2차원 리스트에 저장 
kid_list = []
base_url = 'https://movie.naver.com'

for i in range(len(a_list)):
    title = td_list[i].select_one('a').text
    score = td_list[i].select_one('em').text
    review = td_list[i].select_one('br').next_sibling.strip()
    link = base_url + td_list[i].select_one('a')['href']
    writer = a_list[i].text
    date = a_list[i].next_sibling.next_sibling
    
    # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
    kid_list.append([title, score, review, link, writer, date])    

In [31]:
len(kid_list)

10

In [32]:
# 4) 2차원 리스트 => 데이타프레임 

df_kid = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
df_kid.sample(2)

,title,score,review,link,writer,date
4,키드,8,이탈리아 은행에서 50만불을 빌려 만든 휴머니스트 찰리 채플린의 장편 감독 데뷔작....,https://movie.naver.com/movie/bi/mi/basic.nave...,tear****,21.03.06
6,키드,8,긴 세월 동안 빛이 바랬을지 몰라도 그 순수함과 따뜻함만은 100년을 뛰어넘는다.,https://movie.naver.com/movie/bi/mi/basic.nave...,hyun****,21.02.25


## 범위를 지정해서 댓글 가져오기 

In [39]:
# 각 페이지의 주소 확인 
# https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=페이지번호
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
for i in range(10):
    print(base_page+str(i+1))

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=10


In [52]:
# 각 페이지의 공통 주소 부분 
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
kid_list = []
base_url = 'https://movie.naver.com'

# 10페이지 저장 
for i in range(10):
    # 실제 페이지 주소 생성 
    url = base_page+str(i+1)
    res = requests.get(url)
    # 200 이면 성공 
    print(res, url) 
    # html 페이지를 텍스트로 저장 
    html_str = res.text
    
    # soup 객체화    
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 제목, 평점, 리뷰, 링크, 날짜와 작성자 텍스트 추출 
    td_list = soup.select('.title')
    a_list = soup.select('.author')

    # 모든 요소를 찾아서 2차원 리스트에 저장
    for j in range(len(a_list)):
        title = td_list[j].select_one('a').text
        score = td_list[j].select_one('em').text
        review = td_list[j].select_one('br').next_sibling.strip()
        link = base_url + td_list[j].select_one('a')['href']
        writer = a_list[j].text
        date = a_list[j].next_sibling.next_sibling

        # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
        kid_list.append([title, score, review, link, writer, date])    
    

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

In [53]:
# 2차원 리스트 => 데이타프레임 
df_kid2 = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
df_kid2.shape

(100, 6)

In [54]:
df_kid2.head(10)

,title,score,review,link,writer,date
0,키드,10,짧고 굵게 행복하고 사랑스럽다,https://movie.naver.com/movie/bi/mi/basic.nave...,jsyb****,21.10.20
1,키드,8,(레미제라블의 장발장과 코제트간 관계가 다소간 롤리타적 색채를 띠는 걸 감안함)어쩌...,https://movie.naver.com/movie/bi/mi/basic.nave...,gold****,21.09.04
2,키드,10,과연 채플린의 영화이다! 이 참담한 포스트 코로나 시대에도 우리에게 웃음과 감동을 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,rjkw****,21.03.23
3,키드,10,생에 첫 무성영화대사 없이 영상과 음악 만으로도 충분한 감동과 해학이 담겨 있다.찰...,https://movie.naver.com/movie/bi/mi/basic.nave...,npr3****,21.03.20
4,키드,8,이탈리아 은행에서 50만불을 빌려 만든 휴머니스트 찰리 채플린의 장편 감독 데뷔작....,https://movie.naver.com/movie/bi/mi/basic.nave...,tear****,21.03.06
5,키드,10,1921년도에 만들어졌다니 놀랍습니다!흑백영화는 처음보는데,https://movie.naver.com/movie/bi/mi/basic.nave...,yeyo****,21.03.01
6,키드,8,긴 세월 동안 빛이 바랬을지 몰라도 그 순수함과 따뜻함만은 100년을 뛰어넘는다.,https://movie.naver.com/movie/bi/mi/basic.nave...,hyun****,21.02.25
7,키드,10,채플린만의 감동 연출법이 빛나는 영화. 진정한 휴머니스트. 희극의 천재.,https://movie.naver.com/movie/bi/mi/basic.nave...,got_****,21.02.23
8,키드,10,역시 찰리채플린? 이 영화를 극장에서 볼수 있는것도 넘 감사했어요. 울더가 웃다가 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,gips****,21.02.20
9,키드,9,찰리채플린 특유의 디테일과 티키타카 무엇보다 아이의 연기력 탑 ^^찰리채플린 영화라...,https://movie.naver.com/movie/bi/mi/basic.nave...,libe****,21.02.14


In [55]:
df_kid2.tail(10)

,title,score,review,link,writer,date
90,키드,10,이거 봤는데 . 완전 재밌었어요ㅎ,https://movie.naver.com/movie/bi/mi/basic.nave...,jane****,10.10.30
91,키드,10,아주 흥미진진하다.,https://movie.naver.com/movie/bi/mi/basic.nave...,ywsu****,10.09.21
92,키드,10,쵝오,https://movie.naver.com/movie/bi/mi/basic.nave...,tall****,10.09.11
93,키드,10,와 짱이다,https://movie.naver.com/movie/bi/mi/basic.nave...,jiwo****,10.08.20
94,키드,10,눈물겨운 어린 시절의 추억이 만들어낸 자화상적 이야기,https://movie.naver.com/movie/bi/mi/basic.nave...,hyuk****,10.08.11
95,키드,8,희극의제왕 찰리채플린!,https://movie.naver.com/movie/bi/mi/basic.nave...,powe****,10.06.01
96,키드,10,최고,https://movie.naver.com/movie/bi/mi/basic.nave...,wjdg****,10.05.03
97,키드,10,와!최고다,https://movie.naver.com/movie/bi/mi/basic.nave...,doct****,10.04.03
98,키드,10,가난하고 구멍난 삶이지만 그 속에 웃음과 사랑이 있구나 !,https://movie.naver.com/movie/bi/mi/basic.nave...,sylo****,10.03.19
99,키드,8,젊은 채플린의 모습과 미국의 어두운 단면을 볼 수 있음.,https://movie.naver.com/movie/bi/mi/basic.nave...,jjyn****,10.03.03


In [56]:
df_kid2.to_csv('output/kid.csv', index=False, encoding='cp949')

In [60]:
# 마지막 페이지인지?  다음 버튼 소스 확인 

# 각 페이지의 공통 주소 부분 
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
page = 0
while(True):
    # 실제 페이지 주소 생성 
    url = base_page+str(page+1)
    res = requests.get(url)
    # 200 이면 성공 
    print(res, url) 
    # html 페이지를 텍스트로 저장 
    html_str = res.text
    
    # soup 객체화    
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 다음 버튼 찾기 
    next_btn = soup.find_all(class_='pg_next')
    # 다음버튼이 없다면 종료     
    if len(next_btn) == 0:
        break
    
    # 페이지 증감
    page += 1
    
print('테스트 종료')        
    

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

In [61]:
# 각 페이지의 공통 주소 부분 
base_page = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
kid_list = []
base_url = 'https://movie.naver.com'
page = 0


while(True):
    # 실제 페이지 주소 생성 
    url = base_page+str(page+1)
    res = requests.get(url)
    # 200 이면 성공 
    print(res, url) 
    # html 페이지를 텍스트로 저장 
    html_str = res.text
    
    # soup 객체화    
    soup = BeautifulSoup(html_str, 'html.parser')
    
    # 제목, 평점, 리뷰, 링크, 날짜와 작성자 텍스트 추출 
    td_list = soup.select('.title')
    a_list = soup.select('.author')
    
    # 모든 요소를 찾아서 2차원 리스트에 저장
    for j in range(len(a_list)):
        title = td_list[j].select_one('a').text
        score = td_list[j].select_one('em').text
        review = td_list[j].select_one('br').next_sibling.strip()
        link = base_url + td_list[j].select_one('a')['href']
        writer = a_list[j].text
        date = a_list[j].next_sibling.next_sibling

        # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
        kid_list.append([title, score, review, link, writer, date])

    
    # 다음 버튼 찾기 
    next_btn = soup.find_all(class_='pg_next')
    # 다음버튼이 없다면 종료     
    if len(next_btn) == 0:
        break
        
    # 페이지 증감
    page += 1
    
print('테스트 종료')        
    

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

In [62]:
# 2차원 리스트 => 데이타프레임 
df_kid3 = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
df_kid3.shape

(144, 6)

In [63]:
df_kid3.head(10)

,title,score,review,link,writer,date
0,키드,10,짧고 굵게 행복하고 사랑스럽다,https://movie.naver.com/movie/bi/mi/basic.nave...,jsyb****,21.10.20
1,키드,8,(레미제라블의 장발장과 코제트간 관계가 다소간 롤리타적 색채를 띠는 걸 감안함)어쩌...,https://movie.naver.com/movie/bi/mi/basic.nave...,gold****,21.09.04
2,키드,10,과연 채플린의 영화이다! 이 참담한 포스트 코로나 시대에도 우리에게 웃음과 감동을 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,rjkw****,21.03.23
3,키드,10,생에 첫 무성영화대사 없이 영상과 음악 만으로도 충분한 감동과 해학이 담겨 있다.찰...,https://movie.naver.com/movie/bi/mi/basic.nave...,npr3****,21.03.20
4,키드,8,이탈리아 은행에서 50만불을 빌려 만든 휴머니스트 찰리 채플린의 장편 감독 데뷔작....,https://movie.naver.com/movie/bi/mi/basic.nave...,tear****,21.03.06
5,키드,10,1921년도에 만들어졌다니 놀랍습니다!흑백영화는 처음보는데,https://movie.naver.com/movie/bi/mi/basic.nave...,yeyo****,21.03.01
6,키드,8,긴 세월 동안 빛이 바랬을지 몰라도 그 순수함과 따뜻함만은 100년을 뛰어넘는다.,https://movie.naver.com/movie/bi/mi/basic.nave...,hyun****,21.02.25
7,키드,10,채플린만의 감동 연출법이 빛나는 영화. 진정한 휴머니스트. 희극의 천재.,https://movie.naver.com/movie/bi/mi/basic.nave...,got_****,21.02.23
8,키드,10,역시 찰리채플린? 이 영화를 극장에서 볼수 있는것도 넘 감사했어요. 울더가 웃다가 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,gips****,21.02.20
9,키드,9,찰리채플린 특유의 디테일과 티키타카 무엇보다 아이의 연기력 탑 ^^찰리채플린 영화라...,https://movie.naver.com/movie/bi/mi/basic.nave...,libe****,21.02.14


In [64]:
df_kid3.tail(10)

,title,score,review,link,writer,date
134,키드,8,글쎄..이런 영화는 별로 라서..,https://movie.naver.com/movie/bi/mi/basic.nave...,qufs****,06.08.20
135,키드,10,영화사에 길이남아 영구히 지구인의 감성에 호소할 시대적 걸작,https://movie.naver.com/movie/bi/mi/basic.nave...,jals****,06.06.06
136,키드,10,단편에다무성 흑백이여도 난 이영화가좋다,https://movie.naver.com/movie/bi/mi/basic.nave...,jing****,06.05.19
137,키드,10,내가 처음 접한 찰리 채플린의 영화,https://movie.naver.com/movie/bi/mi/basic.nave...,won1****,06.02.07
138,키드,10,잭키쿠건(아이)의 연기는 일품이다.,https://movie.naver.com/movie/bi/mi/basic.nave...,wndh****,05.10.12
139,키드,8,신파도 맛깔스러운 채플린.,https://movie.naver.com/movie/bi/mi/basic.nave...,tjsd****,05.09.24
140,키드,10,그 당시의 시대적 상황을 웃음과 감동으로 잘 표현한 수작,https://movie.naver.com/movie/bi/mi/basic.nave...,hmr3****,04.10.17
141,키드,10,감동적인 드라마.,https://movie.naver.com/movie/bi/mi/basic.nave...,pby1****,04.10.10
142,키드,10,웃음으로 포장한 가슴 아픈 비극,https://movie.naver.com/movie/bi/mi/basic.nave...,litt****,04.09.16
143,키드,9,따뜻한 아빠 채플린과 귀여운 아이,https://movie.naver.com/movie/bi/mi/basic.nave...,sjzs****,03.12.26


## 코드 개선 => 함수로 

In [67]:
def save_df(base_page_url):
    # 각 페이지의 공통 주소 부분 
    base_page = base_page_url
    kid_list = []
    base_url = 'https://movie.naver.com'
    page = 0


    while(True):
        # 실제 페이지 주소 생성 
        url = base_page+str(page+1)
        res = requests.get(url)
        # 200 이면 성공 
        print(res, url) 
        # html 페이지를 텍스트로 저장 
        html_str = res.text

        # soup 객체화    
        soup = BeautifulSoup(html_str, 'html.parser')

        # 제목, 평점, 리뷰, 링크, 날짜와 작성자 텍스트 추출 
        td_list = soup.select('.title')
        a_list = soup.select('.author')

        # 모든 요소를 찾아서 2차원 리스트에 저장
        for j in range(len(a_list)):
            title = td_list[j].select_one('a').text
            score = td_list[j].select_one('em').text
            review = td_list[j].select_one('br').next_sibling.strip()
            link = base_url + td_list[j].select_one('a')['href']
            writer = a_list[j].text
            date = a_list[j].next_sibling.next_sibling

            # 1차원 리스트로 생성한 후 다시 리스트에 삽입      
            kid_list.append([title, score, review, link, writer, date])


        # 다음 버튼 찾기 
        next_btn = soup.find_all(class_='pg_next')
        # 다음버튼이 없다면 종료     
        if len(next_btn) == 0:
            break

        # 페이지 증감
        page += 1

    print('테스트 종료')   
    df = pd.DataFrame(kid_list, columns=['title', 'score', 'review', 'link', 'writer', 'date'])
    
    return df

In [68]:
# kid 영화 
base_page_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page='
save_df(base_page_url)

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=13558&target=after&page=9
<Response [200]> https://mov

,title,score,review,link,writer,date
0,키드,10,짧고 굵게 행복하고 사랑스럽다,https://movie.naver.com/movie/bi/mi/basic.nave...,jsyb****,21.10.20
1,키드,8,(레미제라블의 장발장과 코제트간 관계가 다소간 롤리타적 색채를 띠는 걸 감안함)어쩌...,https://movie.naver.com/movie/bi/mi/basic.nave...,gold****,21.09.04
2,키드,10,과연 채플린의 영화이다! 이 참담한 포스트 코로나 시대에도 우리에게 웃음과 감동을 ...,https://movie.naver.com/movie/bi/mi/basic.nave...,rjkw****,21.03.23
3,키드,10,생에 첫 무성영화대사 없이 영상과 음악 만으로도 충분한 감동과 해학이 담겨 있다.찰...,https://movie.naver.com/movie/bi/mi/basic.nave...,npr3****,21.03.20
4,키드,8,이탈리아 은행에서 50만불을 빌려 만든 휴머니스트 찰리 채플린의 장편 감독 데뷔작....,https://movie.naver.com/movie/bi/mi/basic.nave...,tear****,21.03.06
...,...,...,...,...,...,...
139,키드,8,신파도 맛깔스러운 채플린.,https://movie.naver.com/movie/bi/mi/basic.nave...,tjsd****,05.09.24
140,키드,10,그 당시의 시대적 상황을 웃음과 감동으로 잘 표현한 수작,https://movie.naver.com/movie/bi/mi/basic.nave...,hmr3****,04.10.17
141,키드,10,감동적인 드라마.,https://movie.naver.com/movie/bi/mi/basic.nave...,pby1****,04.10.10
142,키드,10,웃음으로 포장한 가슴 아픈 비극,https://movie.naver.com/movie/bi/mi/basic.nave...,litt****,04.09.16


In [69]:
# 고양이를 부탁해 영화 
base_page_url = 'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page='
df_cat = save_df(base_page_url)

<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=1
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=2
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=3
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=4
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=5
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=6
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=7
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=8
<Response [200]> https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=31607&target=after&page=9
<Response [200]> https://mov

In [71]:
df_cat.tail(10)

,title,score,review,link,writer,date
727,고양이를 부탁해,8,내나이 20살에 가슴으로 먼저 다가온 영화다..,https://movie.naver.com/movie/bi/mi/basic.nave...,skfk****,04.02.06
728,고양이를 부탁해,10,공허함. 그저 평범한 나의 이야기. 그래 그래서인지 더 눈물이 난다.,https://movie.naver.com/movie/bi/mi/basic.nave...,sos4****,04.01.26
729,고양이를 부탁해,8,일상적인 컨셉을 여성특유의 감각으로 잘 잡은 영화,https://movie.naver.com/movie/bi/mi/basic.nave...,will****,04.01.09
730,고양이를 부탁해,10,각 주인공에게 비중을 두는게 어려운일인데 잘 해냈다. 이요원의 캐릭터가굿,https://movie.naver.com/movie/bi/mi/basic.nave...,kyy1****,04.01.05
731,고양이를 부탁해,10,주변에서일어날것만같은 이야기들......감동,https://movie.naver.com/movie/bi/mi/basic.nave...,simi****,04.01.04
732,고양이를 부탁해,7,스무살의 삶. 미화된 영화 속 얘기가 아니라 가장 현실에 가까웠다.,https://movie.naver.com/movie/bi/mi/basic.nave...,self****,03.12.23
733,고양이를 부탁해,9,영원할것만같던 시간들이 현실에서 소리없이사라져가고있었다..,https://movie.naver.com/movie/bi/mi/basic.nave...,simi****,03.12.23
734,고양이를 부탁해,5,남들이 좋다고 한다고 해서 나에게까지 좋으라는 법은 없는 영화,https://movie.naver.com/movie/bi/mi/basic.nave...,patr****,03.12.16
735,고양이를 부탁해,6,적당히 진지하다. 그 이상도 그 이하도 없다.,https://movie.naver.com/movie/bi/mi/basic.nave...,ily6****,03.12.16
736,고양이를 부탁해,9,스무살라면 한번쯤 봐야할 영화.,https://movie.naver.com/movie/bi/mi/basic.nave...,ihea****,03.12.03
